In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import abp_sim

# ------------------------
# Fixed simulation parameters
# ------------------------
N = 500
radius = 1.0
phi = 0.5
dt = 0.01
steps = 20000
thermalization_steps = 5000

L = np.sqrt(N * np.pi * radius**2 / phi)

# Peclet numbers to scan
Pe_list = [5, 10, 20, 40, 80, 120]

# ------------------------
# Loop over Peclet numbers
# ------------------------
for Pe in Pe_list:

    print(f"\nRunning simulation for Pe = {Pe}")

    # ------------------------
    # Initialize positions & orientations
    # ------------------------
    positions = np.random.rand(N, 2) * L
    thetas = 2 * np.pi * np.random.rand(N)

    # ------------------------
    # Create system
    # ------------------------
    system = abp_sim.System(N, L, Pe)
    system.initialize_particles(
        positions.tolist(),
        thetas.tolist()
    )

    particles = system.get_particles()
    evolver = abp_sim.Evolver(system)

    # ------------------------
    # Thermalization
    # ------------------------
    for _ in range(thermalization_steps):
        evolver.step()

    # ------------------------
    # Main simulation
    # ------------------------
    for _ in range(steps - thermalization_steps):
        evolver.step()

    # ------------------------
    # Extract positions
    # ------------------------
    pos = np.array([p.position for p in particles])

    # ------------------------
    # Snapshot plot (MOST IMPORTANT)
    # ------------------------
    plt.figure(figsize=(5, 5))
    plt.scatter(pos[:, 0], pos[:, 1], s=6)
    plt.title(f"ABPs at Pe = {Pe}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim(0, L)
    plt.ylim(0, L)
    plt.gca().set_aspect("equal")
    plt.tight_layout()
    plt.show()

    # ------------------------
    # Density histogram (optional but very useful)
    # ------------------------
    H, _, _ = np.histogram2d(
        pos[:, 0], pos[:, 1],
        bins=40,
        range=[[0, L], [0, L]]
    )

    plt.figure(figsize=(5, 4))
    plt.imshow(H.T, origin="lower", cmap="inferno",
               extent=[0, L, 0, L])
    plt.colorbar(label="Local density")
    plt.title(f"Density field at Pe = {Pe}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.tight_layout()
    plt.show()
